In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [35]:
import warnings
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_absolute_error

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.eval_measures import mse, rmse

from scipy import stats

from sqlalchemy import create_engine

import seaborn as sns
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [5]:
warnings.filterwarnings("ignore")

postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
house_prices_df = pd.read_sql_query("select * from houseprices", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


<IPython.core.display.Javascript object>

In [36]:
class Markov:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.model = LinearRegression()
        self.model.fit(X, y)
        self.predictions = self.model.predict(X)
        self.errors = self.y - self.predictions
        self.X_const = sm.add_constant(X)
        self.lm_results = sm.OLS(y, self.X_const).fit()
        self.coefs = self.model.coef_
        self.intercepts = self.model.intercept_

    def plot_linearity(self):
        count = 1
        plt.figure(figsize=(25, 15))
        cols = self.X.columns
        for col in self.X.columns:
            plt.subplot(len(cols) / 3, len(cols) / 3, count)
            plt.scatter(self.X[col], self.predictions)
            plt.xlabel(col)
            plt.ylabel("target")
            count += 1

        plt.tight_layout()
        plt.show()

    def plot_homoscedasticity(self):
        plt.scatter(self.predictions, self.errors)
        plt.xlabel("Predicted")
        plt.ylabel("Residual")
        plt.axhline(y=0)
        plt.title("Residual vs. Predicted")
        plt.show()

    def b_pagan(self):
        _, lmp, _, fp = het_breuschpagan(lm_results.resid, X)

        return lmp, fp

    def get_vifs(self):
        vifs = []
        for i in range(self.X_const.shape[1]):
            vif = variance_inflation_factor(self.X_const.values, i)
            vifs.append(vif)

        return pd.Series(vifs, index=X_const.columns)

    def plot_errors(self):
        plt.plot(self.errors)
        plt.show()

    def plot_errors_acf(self):
        acf_data = acf(self.errors)

        plt.plot(acf_data[1:])
        plt.show()

    def plot_error_normality(self):
        qqplot(lm_results.resid, line="s")
        plt.show()

        plt.hist(lm_results.resid)
        plt.show()

    def shapiro_wilkes(self):
        return stats.shapiro(self.lm_results.resid)

<IPython.core.display.Javascript object>

In [30]:
def clean_house_prices_df(house_prices_df):
    house_prices_df = house_prices_df.set_index("id")

    # Log transfrom dataframe
    log_df = house_prices_df.copy()
    log_df["log_saleprice"] = np.log(log_df["saleprice"])
    log_df = log_df.drop(columns=["saleprice"])

    # Keep only top correlated columns
    corr_df = log_df.corr()[["log_saleprice"]]
    corr_df.columns = ["corr"]
    corr_df["abs_corr"] = corr_df.abs()
    top_corrs = corr_df.sort_values("abs_corr", ascending=False).head(10)
    num_cols = log_df.select_dtypes("number").columns
    keep_cols = top_corrs.index
    drop_cols = [c for c in num_cols if c not in keep_cols]
    log_df = log_df.drop(columns=drop_cols)
    log_df

    drop_cols = ["poolqc", "alley", "fence", "fireplacequ", "miscfeature"]
    log_df = log_df.drop(columns=drop_cols)
    log_df = log_df.dropna()

    cat_cols = log_df.select_dtypes("O").copy()
    keep_cols = ["exterqual", "bsmtqual", "kitchenqual", "centralair"]
    drop_cols = [c for c in cat_cols.columns if c not in keep_cols]

    log_df = log_df.drop(columns=drop_cols)

    # Encode centralair as binary
    log_df["centralair"] = (log_df["centralair"] == "Y").astype(int)

    # Encode qual columns as ordinal
    quality_map = {"Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}
    log_df[["exterqual", "bsmtqual", "kitchenqual"]] = log_df[
        ["exterqual", "bsmtqual", "kitchenqual"]
    ].replace(quality_map)
    
    log_df= log_df.drop(columns=["fullbath", "garagearea", "totalbsmtsf", "exterqual"])

    return log_df




<IPython.core.display.Javascript object>

In [31]:
houses = clean_houses_data(house_prices_df)
X = houses.drop(columns=["log_saleprice"])
y = houses["log_saleprice"]
house_markov = Markov(X, y)

<IPython.core.display.Javascript object>

In [32]:
houses.head()

,overallqual,yearbuilt,yearremodadd,bsmtqual,centralair,firstflrsf,grlivarea,kitchenqual,garagecars,log_saleprice
id,,,,,,,,,,
1,7,2003,2003,3,1,856,1710,3,2,12.247694
2,6,1976,1976,3,1,1262,1262,2,2,12.109011
3,7,2001,2002,3,1,920,1786,3,2,12.317167
4,7,1915,1970,2,1,961,1717,3,3,11.849398
5,8,2000,2000,3,1,1145,2198,3,3,12.429216


<IPython.core.display.Javascript object>

In [33]:
houses.shape

(1338, 10)

<IPython.core.display.Javascript object>

In [34]:
lm_results = house_markov.lm_results
lm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          log_saleprice   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     666.6
Date:                Wed, 08 Apr 2020   Prob (F-statistic):               0.00
Time:                        19:50:10   Log-Likelihood:                 550.32
No. Observations:                1338   AIC:                            -1081.
Df Residuals:                    1328   BIC:                            -1029.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            6.2782      0.674      9.311      0.000       4.955       7.601
overallqual      0.0806      0.006     13.883      0.000       0.069       0.092
yearbuilt        0.0009      0.000      3.643      0.000       0.000       0.001
yearremodadd     0.0012      0.000      3.824      0.000       0.001       0.002
bsmtqual         0.0474      0.011      4.365      0.000       0.026       0.069
centralair       0.1884      0.023      8.194      0.000       0.143       0.233
firstflrsf       0.0001   1.44e-05      8.980      0.000       0.000       0.000
grlivarea        0.0002    1.2e-05     18.027      0.000       0.000       0.000
kitchenqual      0.0502      0.010      4.933      0.000       0.030       0.070
garagecars       0.0691      0.010      7.212      0.000       0.050       0.088
==============================================================================
Omnibus:                      841.968   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            29202.077
Skew:                          -2.367   Prob(JB):                         0.00
Kurtosis:                      25.392   Cond. No.                     5.24e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.24e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>